In [1]:
# use lod env
import xml.etree.ElementTree as ET
import re
import pandas as pd
import os
import geopandas as gpd
from shapely.geometry import Polygon

## Read in LoD2 data from gml files, clean up and save into tables

Some parts of the extraction code are taken from: https://git.informatik.uni-leipzig.de/scads/cut/cut_buildings_data-api/-/tree/develop?ref_type=heads , commit SHA: 0621cea73f20f1719d07e16fa2c30c6c7a001d25

This is marked with "taken from existing code"

In [2]:
# important directories
data_dir = "../../data"
gml_dir = data_dir + "/gml"
dataframes_dir = data_dir + "/dataframes"

In [ ]:
# in the end the dataframes should look like this:

######## df 1 - level of parts (part_id should be unique)
# building_id  part_id  street  house_nr  hnr_add  district  roof_form  building_height  ground_surface

######## df 2 - level of wall, only walls (wall_id should be unique)
# part_id  wall_id  surface_coordinates  features_to_come..

######## df 3 - level of wall, only roof surfaces (surface_id should be unique)
# part_id  surface_id  surface_coordinates  evtl_features

### Store all buildings in one dataframe

In [3]:
# taken from existing code
def convert_coordinate_pos_list_to_tuple(coords_as_text):
    ''' 
    Converts a string of coordinates into a list of tuples, each containing three values.
    '''
    # convert coordinates from string to float
    coords_list = [float(coord) for coord in coords_as_text.split()] 
    # save coordinates as tuples of 3
    tuple_list = [(coords_list[i], coords_list[i+1], coords_list[i+2]) for i in range(0, len(coords_list), 3)] 
    return tuple_list


# taken from existing code
def extract_building_adress(building, nspace):
    ''' 
    For a single building, extract adress and convert from one string to 3 values: street, house_nr and hnr_add (addition to house nr)
    '''
    adress = building.find('gen:stringAttribute[@name="Adresse"]/gen:value', nspace).text
    # sometimes adress contains several streets seperatet by ","
    # if more than one adress, take first one
    if "," in adress:
        adress = adress.split(',')[0]
       
    #no adress
    if adress.isspace(): 
        street = ""
        house_nr = ""
        hnr_add = ""

    else:
        for idx, char in enumerate(adress):
            if char.isdigit():
                house_nr = adress[idx:]
                street = adress[:idx-1] # remove space at end of string with -1
                hnr_add = ""
                break
        try: # check if house nr has additional characters
            house_nr = int(house_nr)
        except ValueError:
            for idx, char in enumerate(house_nr):
                if not char.isdigit():
                    house_nr = house_nr[:idx]
                    hnr_add = char

    return street, house_nr, hnr_add

# partly taken from existing code, added extraction of part id and include extraction of wall and roof ids for own dataframes df2 and df3
def extract_building_properties(building, is_part, nspace):
    ''' 
    for one building, extract building id, height, roof form, wall surfaces (list of dictionary items including wall id, part id and wall surface coordinates), roof surfaces (list of dictionary items including roof id, part id and roof surface coordinates), ground surface and adress
    '''
    ### ID
    # if part, get part_id, else take main building id and add "_main"
    if is_part:
        part_id = building.attrib['{http://www.opengis.net/gml}id']
    else:
        part_id = building.attrib['{http://www.opengis.net/gml}id'] + "_main"

    ### BUILDING HEIGHT
    if is_part:
        try:    
            building_height = float(building.find('gen:stringAttribute[@name="GebaeudeTeilHoehe"]/gen:value', nspace).text) #height attribute has different name if is building part
        except AttributeError:
            building_height = 0.0
    else:
        try:    
            building_height = float(building.find('gen:stringAttribute[@name="GebaeudeHoehe"]/gen:value', nspace).text)
        except AttributeError:
            building_height = 0.0

    ### ROOF FORM
    try:
        roof_form = building.find('gen:stringAttribute[@name="Dachform"]/gen:value', nspace).text
    except AttributeError:
        roof_form = ""
    
    ### WALL SURFACES
    # make own list for those 
    wall_surfaces = []
    wall_surface_items = building.findall("bldg:boundedBy/bldg:WallSurface", nspace)
    if wall_surface_items != []:
        for wall_item in wall_surface_items:
            # get wall_id and coordinates of surface
            wall_surface_coords = convert_coordinate_pos_list_to_tuple(wall_item.find('.//gml:posList', nspace).text)
            wall_surface_id = wall_item.attrib['{http://www.opengis.net/gml}id']
            wall_surfaces.append({'part_id': part_id, 'wall_id': wall_surface_id, 'surface_coordinates': wall_surface_coords})

    ### ROOF SURFACES
    roof_surfaces = []
    roof_surface_items = building.findall("bldg:boundedBy/bldg:RoofSurface", nspace)
    if roof_surface_items != []:
        for roof_item in roof_surface_items:
            # get surface_id and coordinates of surface
            roof_surface_coords = convert_coordinate_pos_list_to_tuple(roof_item.find('.//gml:posList', nspace).text)
            roof_surface_id = roof_item.attrib['{http://www.opengis.net/gml}id']
            roof_surfaces.append({'part_id': part_id, 'surface_id': roof_surface_id, 'surface_coordinates': roof_surface_coords})

    ### GROUND SURFACE
    ground_surface = building.find("bldg:boundedBy/bldg:GroundSurface", nspace)

    ground_surface_tuple_list = []
    if ground_surface is not None:
        ground_surface_tuple_list = convert_coordinate_pos_list_to_tuple(ground_surface.find('.//gml:posList', nspace).text)

    ### ADRESS
    street, house_nr, hnr_add = extract_building_adress(building, nspace)

    return part_id, roof_form, roof_surfaces, building_height, ground_surface_tuple_list, wall_surfaces, street, house_nr, hnr_add

# partly taken from existing code, changed to extract the part-information seperately to the wall and roof information
def read_lod2_file(lod2_file):
    ''' 
    parse gml file and for each building: extract building id, district, buildings parts and for each main building and part: use previous function to extract surfaces and other attributes. Transform to 3 different dataframes, df1 for parts, df2 for walls, df3 for roof surfaces.
    '''

    nspace = {'gml': 'http://www.opengis.net/gml', 
              'core': "http://www.opengis.net/citygml/1.0", 
              'bldg':"http://www.opengis.net/citygml/building/1.0", 
              'gen': "http://www.opengis.net/citygml/generics/1.0"}

    xml_file = ET.parse(lod2_file)
    root = xml_file.getroot()

    all_city_objs = root.findall('core:cityObjectMember', nspace)

    print("nr of city_objects:", len(all_city_objs))
    df1_parts_list = []   
    df2_walls_list = []  
    df3_roofs_list = []  

    for idx, object in enumerate(all_city_objs):
        #print(f"Reading xml file: Entry {idx+1} of {len(all_city_objs)}")

        building = object.find('bldg:Building', nspace)

        # simple check if there is any geometry in there (ground surface, walls, roofs, buildings parts with geometry... always starting with <boundedBy>)
        # ignore building if there is not
        if len(building.findall('.//bldg:boundedBy', nspace)) > 0:
            
            ### BUILDING ID
            building_id = building.attrib['{http://www.opengis.net/gml}id']

            ### BUILDING PARTS
            building_parts = building.findall('bldg:consistsOfBuildingPart', nspace)
            has_building_parts = len(building_parts) > 0

            ### DISTRICT
            district = re.search(r'_([^_]+)\.gml', lod2_file).group(1)

            # get building properties and wall and roof surfaces and add to corresponding lists
            part_id, roof_form, roof_surfaces, building_height, ground_surface_coords, wall_surfaces, street, house_nr, hnr_add = extract_building_properties(building, False, nspace)
            # add part to df1
            result_item = {'building_id': building_id, 'part_id': part_id, 'has_building_parts': has_building_parts, 'is_part': False,  'street': street,'house_nr': house_nr, 'hnr_add': hnr_add, 'district': district, 'roof_form': roof_form, 'building_height': building_height, 'ground_surface': ground_surface_coords}
            df1_parts_list.append(result_item)
            # add walls and roof surfaces (several per building) to existing lists
            df2_walls_list = df2_walls_list + wall_surfaces
            df3_roofs_list = df3_roofs_list + roof_surfaces

            # do same for each building part
            if has_building_parts:
                for building_part in building_parts:
                    building_part = building_part.find('bldg:BuildingPart', nspace)
                    part_id, roof_form, roof_surfaces, building_height, ground_surface_coords, wall_surfaces, street, house_nr, hnr_add = extract_building_properties(building_part, True, nspace)
                    result_item = {'building_id': building_id, 'part_id': part_id, 'has_building_parts': has_building_parts, 'is_part': True, 'street': street,'house_nr': house_nr, 'hnr_add': hnr_add, 'district': district, 'roof_form': roof_form, 'building_height': building_height, 'ground_surface': ground_surface_coords}
                    df1_parts_list.append(result_item)
                    df2_walls_list = df2_walls_list + wall_surfaces
                    df3_roofs_list = df3_roofs_list + roof_surfaces
    
    df1_parts = pd.DataFrame(df1_parts_list)
    df2_walls = pd.DataFrame(df2_walls_list)
    df3_roofs = pd.DataFrame(df3_roofs_list)

    return df1_parts, df2_walls, df3_roofs


In [ ]:
# runs 4-5min
files_list = os.listdir(gml_dir)

# concatenate all single-district files
# if first file, initialize dataframes, afterwards concatenate to existing dataframes
first = True

for file in files_list:
    if file.endswith(".gml"):
        print(file)
        # extract dataframes for this district
        df1_parts, df2_walls, df3_roofs = read_lod2_file(gml_dir + "/" + file)
        
        # initialize final dataframe
        if first:
            first = False
            df1_parts_all = df1_parts
            df2_walls_all = df2_walls
            df3_roofs_all = df3_roofs
        # concatenate to final dataframe
        else:
            df1_parts_all = pd.concat([df1_parts_all, df1_parts])
            df2_walls_all = pd.concat([df2_walls_all, df2_walls])
            df3_roofs_all = pd.concat([df3_roofs_all, df3_roofs])


GEB3D_Leipzig_LoD2_63_Gruenau-Siedlung.gml
nr of city_objects: 3022
GEB3D_Leipzig_LoD2_27_Engelsdorf.gml
nr of city_objects: 5261
GEB3D_Leipzig_LoD2_10_Schoenefeld-Abtnaundorf.gml
nr of city_objects: 1891
GEB3D_Leipzig_LoD2_32_Probstheida.gml
nr of city_objects: 2630
GEB3D_Leipzig_LoD2_20_Neustadt-Neuschoenefeld.gml
nr of city_objects: 1017
GEB3D_Leipzig_LoD2_40_Suedvorstadt.gml
nr of city_objects: 1971
GEB3D_Leipzig_LoD2_73_Leutzsch.gml
nr of city_objects: 3004
GEB3D_Leipzig_LoD2_28_Baalsdorf.gml
nr of city_objects: 1924
GEB3D_Leipzig_LoD2_06_Zentrum-Nord.gml
nr of city_objects: 864
GEB3D_Leipzig_LoD2_02_Zentrum-Suedost.gml
nr of city_objects: 1034
GEB3D_Leipzig_LoD2_92_Gohlis-Nord.gml
nr of city_objects: 1379
GEB3D_Leipzig_LoD2_12_Mockau-Sued.gml
nr of city_objects: 1305
GEB3D_Leipzig_LoD2_14_Thekla.gml
nr of city_objects: 4020
GEB3D_Leipzig_LoD2_50_Schleussig.gml
nr of city_objects: 1889
GEB3D_Leipzig_LoD2_24_Paunsdorf.gml
nr of city_objects: 2005
GEB3D_Leipzig_LoD2_42_Marienbrunn.g

In [18]:
df1_parts_all = df1_parts_all.reset_index(drop = True)
df1_parts_all

,building_id,part_id,has_building_parts,is_part,street,house_nr,hnr_add,district,roof_form,building_height,ground_surface
0,DESN_000WFP8,DESN_000WFP8_main,True,False,Erlanger Straße,28,,Gruenau-Siedlung,Pultdach,4.38,"[(311035.68, 5688001.348, 119.622), (311030.45..."
1,DESN_000WFP8,DESN_1389772836945_13701754,True,True,Erlanger Straße,28,,Gruenau-Siedlung,Mischform,7.89,"[(311036.776, 5687996.602, 119.626), (311031.5..."
2,DESN_0002JBA,DESN_0002JBA_main,True,False,Lichtenfelser Straße,15,,Gruenau-Siedlung,Walmdach,8.68,"[(311357.95, 5687941.966, 120.262), (311360.34..."
3,DESN_0002JBA,DESN_1389772836945_13702111,True,True,Lichtenfelser Straße,15,,Gruenau-Siedlung,Flachdach,4.73,"[(311350.278, 5687935.291, 120.255), (311348.0..."
4,DESN_0017TMK,DESN_0017TMK_main,True,False,Bamberger Straße,29,,Gruenau-Siedlung,Walmdach,6.68,"[(311912.267, 5687965.581, 124.354), (311897.8..."
...,...,...,...,...,...,...,...,...,...,...,...
207207,DESN_0007WEF,DESN_0007WEF_main,False,False,Falterstraße,40,,Heiterblick,Satteldach,10.33,"[(324398.321, 5692808.349, 127.941), (324393.8..."
207208,DESN_0006VB7,DESN_0006VB7_main,False,False,Goldene Hufe,23,,Heiterblick,Flachdach,2.88,"[(324332.827, 5692994.871, 128.691), (324338.7..."
207209,DESN_71085,DESN_71085_main,False,False,Goldene Hufe,25,,Heiterblick,Flachdach,3.22,"[(324326.185, 5692996.398, 128.842), (324324.4..."
207210,DESN_0010UAC,DESN_0010UAC_main,False,False,Torgauer Straße,279,,Heiterblick,Satteldach,2.49,"[(323139.308, 5693719.885, 129.348), (323142.2..."


In [19]:
df2_walls_all = df2_walls_all.reset_index(drop = True)
df2_walls_all

,part_id,wall_id,surface_coordinates
0,DESN_000WFP8_main,UUID_0bbcf935-b8f5-49d6-8931-af23caaa2903,"[(311035.68, 5688001.348, 123.998), (311030.45..."
1,DESN_000WFP8_main,UUID_d94fd64c-53e3-4563-9fe1-3a9b840fb214,"[(311030.45, 5688000.177, 123.998), (311028.45..."
2,DESN_000WFP8_main,UUID_78e9d3b0-4fdd-4fe2-94fe-06ce701ffa86,"[(311028.453, 5688008.816, 122.198), (311033.6..."
3,DESN_000WFP8_main,UUID_41f1c6a6-8857-4ce3-b0f5-3eb1aaa5bc54,"[(311033.653, 5688009.9, 122.198), (311035.68,..."
4,DESN_1389772836945_13701754,UUID_465e5cc3-707b-48e4-a7ee-81a03a9e2ee6,"[(311036.776, 5687996.602, 124.441), (311031.5..."
...,...,...,...
1084355,DESN_{BC4A4139-375F-42BC-BC74-F2B4096D37B9}_main,fme-gen-d478c53a-fc5b-458e-bb9b-2f62cf9c1e23,"[(324095.476, 5692964.556, 134.086), (324088.1..."
1084356,DESN_{BC4A4139-375F-42BC-BC74-F2B4096D37B9}_main,fme-gen-aa9df72b-5aee-490f-b88c-d6383c6f1787,"[(324088.163, 5692958.332, 134.086), (324081.6..."
1084357,DESN_{BC4A4139-375F-42BC-BC74-F2B4096D37B9}_main,fme-gen-f74ccd45-45d0-4f2e-9327-b37991fc8ac8,"[(324081.677, 5692965.971, 134.086), (324088.9..."
1084358,DESN_{BC4A4139-375F-42BC-BC74-F2B4096D37B9}_main,fme-gen-332c0451-e574-4184-9536-80db5b3e2ae0,"[(324088.163, 5692958.332, 134.086), (324095.4..."


In [20]:
df3_roofs_all = df3_roofs_all.reset_index(drop = True)
df3_roofs_all

,part_id,surface_id,surface_coordinates
0,DESN_000WFP8_main,UUID_8820ec37-07fd-4638-9068-069e26cffad2,"[(311033.653, 5688009.9, 122.198), (311028.453..."
1,DESN_1389772836945_13701754,UUID_36080bda-7556-4cd6-98b0-d85824704302,"[(311030.45, 5688000.177, 124.441), (311031.54..."
2,DESN_1389772836945_13701754,UUID_63b276a5-3fce-4b36-a8ee-1d2c6e6cd38d,"[(311031.545, 5687995.439, 124.441), (311036.7..."
3,DESN_1389772836945_13701754,UUID_04a7246c-dd21-4ceb-96d8-4b0fd7cd2573,"[(311035.68, 5688001.348, 124.441), (311030.45..."
4,DESN_0002JBA_main,UUID_da3b67ac-ec03-4a6c-9ad1-53a5e38fb0c4,"[(311351.694, 5687928.908, 124.984), (311360.3..."
...,...,...,...
344263,DESN_71085_main,UUID_31f2adfc-fa76-4474-84d4-1636f434020a,"[(324331.407, 5693000.902, 132.059), (324329.6..."
344264,DESN_0010UAC_main,UUID_88517093-74a7-4736-8e21-829b4b51c588,"[(323142.286, 5693714.619, 131.483), (323139.3..."
344265,DESN_0010UAC_main,UUID_a13fc74a-f941-4ead-8057-4fc8577e464d,"[(323136.705, 5693718.41, 131.483), (323139.68..."
344266,DESN_{BC4A4139-375F-42BC-BC74-F2B4096D37B9}_main,fme-gen-fb1b9642-b20c-40b1-bb2a-2705b3cd3a11,"[(324095.476, 5692964.556, 134.086), (324088.9..."


### Test important requirements

In [21]:
# key rows of each dataframe should be unique
print("part ids of df1 are unique:", df1_parts_all["part_id"].is_unique)
print("wall ids of df2 are unique:", df2_walls_all["wall_id"].is_unique)
print("surface ids of df3 are unique:", df3_roofs_all["surface_id"].is_unique)

part ids of df1 are unique: True
wall ids of df2 are unique: True
surface ids of df3 are unique: True


In [22]:
# how many buildings do we have?
df1_parts_all['building_id'].nunique()

154960

### clean data

In [23]:
# 1. remove double coordinates (except for the first == last coordinate tuple)
def remove_double_coordinates(df, col_name):
    ''' 
    remove all duplicate coordinates from lists with coordinate tuples. The function goes through all coordinate tuples lists in the given dataframe in column col_name. Returns dataframe with corrected coordinate lists (without duplicates)    
    '''
    counter = 0
    for index, row in df.iterrows():
        coords = row[col_name]
        if coords:  # only if list not empty
            coords_new = []
            [coords_new.append(item) for item in coords if item not in coords_new]
            coords_new.append(coords[0])
            if coords_new != coords:
                df.at[index, col_name] = coords_new
                counter += 1

    print(counter, "coordinates have been changed")

    return df

df1_parts_all = remove_double_coordinates(df1_parts_all, "ground_surface")
df2_walls_all = remove_double_coordinates(df2_walls_all, "surface_coordinates")
df3_roofs_all = remove_double_coordinates(df3_roofs_all, "surface_coordinates")

29 coordinates have been changed
7489 coordinates have been changed
590 coordinates have been changed


In [24]:
# 2. check if there are only 3 or less coordinates left somewhere (as first == last this would result in a line, not area)
def less_than_4_coords(df, col_name):
    '''
    checks for all coordinate tuple lists in a df in a column col_name, how many lists have less than 4 coordinate tuples and prints result
    '''
    counter = 0
    for index, row in df.iterrows():
        coords = row[col_name]
        if len(coords) < 4: 
            counter += 1

    print(counter, "coordinates have less than 4 coordinates")

less_than_4_coords(df1_parts_all, "ground_surface")
less_than_4_coords(df2_walls_all, "surface_coordinates")
less_than_4_coords(df3_roofs_all, "surface_coordinates")

# for ground_surfaces: leave it in there for the moment (might still be important for reference position)

1616 coordinates have less than 4 coordinates
0 coordinates have less than 4 coordinates
0 coordinates have less than 4 coordinates


In [5]:
# 3. check if some walls are duplicate: their parts have the exact same ground surface and both walls have the same wall surface (for some reason, sometimes the main building part is additionally modelled as seperate part OR a building is modelled twice with different building ids)
df1_parts_temp = df1_parts_all.copy()
df1_parts_temp["ground_surface"] = [Polygon(ground_surface) for ground_surface in df1_parts_temp["ground_surface"]]
df1_parts_gdf = gpd.GeoDataFrame(df1_parts_temp, geometry = "ground_surface")

def get_wall_ids_to_remove_from_df2(part_id1, part_id2):
    '''
    Gets two parts with the same ground surface as input. For all walls in part1, look if we have the exact same wall in part2, if yes, return the ids of the second walls for each duplicate, which are to be removed
    '''
    ids_to_return = []
    for _, wall_row in df2_walls_all[df2_walls_all["part_id"] == part_id1].iterrows():
        for _, wall2_row in df2_walls_all[df2_walls_all["part_id"] == part_id2].iterrows():
            if wall_row["surface_coordinates"] == wall2_row["surface_coordinates"]:
                # removing second wall ensures removing the wall of the part and not of the main building if both are the same, as the main building always comes first in df2
                ids_to_return.append(wall2_row["wall_id"])

    return ids_to_return

# check which parts have the same groundsurface
counter_same_ground_surface = 0
# accumulate all wall ids that should be removed
ids_to_remove = []
for index, row in df1_parts_gdf.iterrows():
    ground = row["ground_surface"]
    # check which other parts are actually close enough to be the same
    neighbours = df1_parts_gdf[df1_parts_gdf["ground_surface"].intersects(ground)]
    for index2, row2 in neighbours.iterrows():
        if index < index2:
            if ground.equals(row2["ground_surface"]):
                counter_same_ground_surface += 1
                ids_to_remove = ids_to_remove + get_wall_ids_to_remove_from_df2(row["part_id"], row2["part_id"])
                
print(counter_same_ground_surface, "pairs of parts have the same ground surface")
print(len(ids_to_remove), "pairs of walls have furthermore the same wall surface")

75 pairs of parts have the same ground surface
90 pairs of walls have furthermore the same wall surface


In [ ]:
# save IDs to remove 
with open(dataframes_dir + "/duplicate_walls_ids.txt", 'w') as file:
    file.write(str(ids_to_remove))


In [7]:
print(len(df2_walls_all), "walls before")
df2_walls_all = df2_walls_all[~df2_walls_all["wall_id"].isin(ids_to_remove)]
print(len(df2_walls_all), "after removing duplicate walls")

1084360 walls before
1084285 after removing duplicate walls


In [30]:
# save as csv
df1_parts_all.to_csv(dataframes_dir + "/df1_parts.csv", index = False)
# add version nr (00) to df2 as there will be several versions along the process
df2_walls_all.to_csv(dataframes_dir + "/df2_walls_00.csv", index = False)
df3_roofs_all.to_csv(dataframes_dir + "/df3_roofs.csv", index = False)